<a href="https://colab.research.google.com/github/dfmunoz11/Modelo_predictivo_SaberPro_2025/blob/main/02_preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# =====================================================
# Preprocesado y limpieza - UDEA AI4ENG 20252
# Pruebas Saber Pro - IA para ingenierías
# =====================================================


In [1]:
# Configurar el entorno

!pip install -q kaggle pandas seaborn matplotlib

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Cargar credenciales de Kaggle
# El archivo kaggle.json se carga desde el pc

from google.colab import files
files.upload() # <- Esta linea nos pide subir el archivo
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
# Descargar los datos

!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!mkdir -p data
!unzip -o udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip -d data

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.38GB/s]
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: data/submission_example.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          


In [5]:
# Descargar los datos

print("Archivos en la carpeta data:")
!ls data

Archivos en la carpeta data:
submission_example.csv	test.csv  train.csv


In [6]:
# Carga del archivo train.csv

train_path = os.path.join("data", "train.csv")
df = pd.read_csv(train_path)

print("Tamaño del dataset:", df.shape)
df.head()

Tamaño del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294



# Conversión de valores ordinales en númericos.
# Estas columnas tienen un orden lógico, por lo que las convierto en números para que los modelos puedan interpretarlas correctamente.


In [7]:
# 4.1 - Valor de matrícula (estimado en millones de pesos)
tuition_average_mapping = {
    "No pagó matrícula": 0.0,
    "Menos de 500 mil": 0.25,
    "Entre 500 mil y menos de 1 millón": 0.75,
    "Entre 1 millón y menos de 2.5 millones": 1.75,
    "Entre 2.5 millones y menos de 4 millones": 3.25,
    "Entre 4 millones y menos de 5.5 millones": 4.75,
    "Entre 5.5 millones y menos de 7 millones": 6.25,
    "Más de 7 millones": 7.5
}
if 'ESTU_VALORMATRICULAUNIVERSIDAD' in df.columns:
    df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map(tuition_average_mapping)

In [8]:
# 4.2 - Estrato de vivienda
housing_strata_mapping_average = {
    "Estrato 1": 1,
    "Estrato 2": 2,
    "Estrato 3": 3,
    "Estrato 4": 4,
    "Estrato 5": 5,
    "Estrato 6": 6,
    "Sin Estrato": None  # Se considera valor faltante
}
if 'FAMI_ESTRATOVIVIENDA' in df.columns:
    df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].map(housing_strata_mapping_average)


In [9]:
# 4.3 - Nivel educativo del padre
education_mapping = {
    "No aplica": None,
    "No sabe": None,
    "Ninguno": 0,
    "Primaria incompleta": 1,
    "Primaria completa": 2,
    "Secundaria (Bachillerato) incompleta": 3,
    "Secundaria (Bachillerato) completa": 4,
    "Educación profesional incompleta": 5,
    "Educación profesional completa": 6,
    "Técnica o tecnológica incompleta": 7,
    "Técnica o tecnológica completa": 8,
    "Postgrado": 9
}
if 'FAMI_EDUCACIONPADRE' in df.columns:
    df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].map(education_mapping)
if 'FAMI_EDUCACIONMADRE' in df.columns:
    df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].map(education_mapping)



# Codificación binaria (Sí/No)



In [10]:
binary_mapping = {"Si": 1, "No": 0}

for col in ['FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO']:
    if col in df.columns:
        df[col] = df[col].map(binary_mapping)

# Codificación de la variable objetivo


In [11]:
target_order = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
if 'RENDIMIENTO_GLOBAL' in df.columns:
    df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(target_order)


# Separación de características y variable objetivo


In [12]:
X = df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'], errors='ignore')
y = df['RENDIMIENTO_GLOBAL']


# Identificación de columnas numéricas y categóricas


In [13]:
numeric_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

print("Columnas numéricas:", len(numeric_columns))
print("Columnas categóricas:", len(categorical_columns))

Columnas numéricas: 5
Columnas categóricas: 14


# Creación del pipeline de preprocesamiento


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_columns),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_columns)
    ],
    remainder='passthrough'  # conserva las columnas que ya procesamos manualmente
)

# Aplicar preprocesamiento
X_cleaned = preprocessor.fit_transform(X)


# Convertir a DataFrame y guardar


In [15]:
X_cleaned_df = pd.DataFrame(X_cleaned)
train_cleaned_df = pd.concat([X_cleaned_df, y.reset_index(drop=True)], axis=1)

train_cleaned_df.to_csv("train_cleaned.csv", index=False)

print("✅ Datos preprocesados guardados en 'train_cleaned.csv'")


✅ Datos preprocesados guardados en 'train_cleaned.csv'


# Descargar el archivo limpio


In [16]:
from google.colab import files
files.download('train_cleaned.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>